# Building a Climate Logger with the Raspberry Pi Sense Hat

Build a simple climate logger using Raspberry Pi with Sense Hat. The logger reads temperature, humidity, and pressure from the sensors, corrects them for heat output from the Raspberry Pi, and stores the data in a sqlite3 database.

In [30]:
from sense_hat import SenseHat
import time
from datetime import datetime
import math
import sqlite3 as sqlite
import types

## Physics

### Saturation vapor pressure
Function "buck_vapor_pressure" uses Buck's formula (equation 8 in [Buck, 1981](http://www.public.iastate.edu/~bkh/teaching/505/arden_buck_sat.pdf)) to compute [saturation pressure](https://en.wikipedia.org/wiki/Vapor_pressure) $P^*_{vapor}$ (in hPa) for water vapor in moist air at temperature $T$ (in °C) and air pressure $P_{air}$ (in hPa):
\begin{equation}
P_{vapor}^* = \left(1.0007+3.46\cdot10^{-6}P_{air}\right)\cdot 6.1121\exp\left(\frac{17.502T}{240.97 +T}\right)
\end{equation}
The equation is valid in the temperature range from -20 °C to 50 °C.

In [31]:
def buck_vapor_pressure(air_pressure, temperature):

    buck_constants = [1.0007, 3.46e-6, 6.1121, 17.502, 240.97]

    assert air_pressure > 0, "buck_vapor_pressure: negative input air pressure"
    assert temperature >= -20, "buck_vapor_pressure: temperature below validity range"
    assert temperature <= 50, "buck_vapor_pressure: temperature above validity range"

    enhancement_factor = buck_constants[0] + buck_constants[1]*air_pressure
    temperature_exponent = buck_constants[3]*temperature/(buck_constants[4]+temperature)

    vapor_pressure = enhancement_factor*buck_constants[2]*math.exp(temperature_exponent)

    assert vapor_pressure > 0, "buck_vapor_pressure: negative output vapor pressure"

    return (vapor_pressure)

### Correction of temperature measurement
Temperatures are measured inside the enclosure of the Sense Hat and Raspberry Pi. They are heavily influenced by heat generated in the circuitry of the Pi and the Hat and are thus higher than ambient temperature; measured temperatures will depend both on heat input and ambient temperature.

The releationship between $T_{meas}$ and $T_{ambient}$ is close to linear. Fit coefficients for a linear law to determine $T_{ambient}$ from $T_{meas}$ can easily be found by measuring temperature pairs using a second (accurate) thermometer. Note that the Pi and Sense Hat need about 30 min - 45 min to reach stable temperature. Heat generation by the circuitry also depends on processor load; temperature should thus be measured while the Climate Logger program is running.

In [32]:
def ambient_temperature(measured_temperature):

    a_fit = 0.8542
    b_fit = -9.675

    ambient_temperature = a_fit*measured_temperature + b_fit

    return (ambient_temperature)

### Correction of pressure measurement
Air can easily enter and leave the enclosure of the Raspberry Pi and Sense hat - no correction required.

In [33]:
def ambient_pressure(measured_pressure):
    return (measured_pressure)

### Correction of relative humidity measurement

Relative humidity is defined as the ratio of actual vapor pressure and saturation vapor pressure:
\begin{equation}
RH = \frac{P_{vapor}}{P_{vapor}^*}
\end{equation}
Just like air, water vapor can easily enter and leave the enclosure of the Raspberry Pi and Sense Hat. The heating effect of the Raspberry Pi is thus an isobaric process, and $P_{Water}$ is the same inside the enclosure and outside.

We can therefore correct relative humidity measurement by computing saturation vapor pressure for the temperature inside the enclosure and outside. Multiplying $RH_{meas}$ by saturation pressure at temperature $T_{meas}$ and air pressure $P_{meas}$ yields $P_{vapor}$; dividing again by saturation pressure at ambient conditions yields $RH_{ambient}$:
\begin{equation}
RH_{ambient} = RH_{meas}\frac{P_{vapor}^*(T_{meas}, P_{meas})}{P_{vapor}^*(T_{ambient}, P_{meas})}
\end{equation}

In [34]:
def ambient_humidity(measured_humidity, measured_temperature, measured_pressure,
                     ambient_temperature, ambient_pressure):

    inside_vapor_pressure = buck_vapor_pressure(measured_pressure, measured_temperature)
    ambient_vapor_pressure = buck_vapor_pressure(ambient_pressure, ambient_temperature)

    ambient_humidity = measured_humidity*inside_vapor_pressure/ambient_vapor_pressure

    return (ambient_humidity)

## Sensor class

Define new class "Sensor" that handles sensor reads and measurement corrections.

The "update" method reads humidity, temperature (from humidity sensor), and pressure, and gets a UTC time stamp for the measurement from systemclock. It then calls the correction function.

In [35]:
class Sensor:
    def __init__(self):
        self.sense = SenseHat()
        self.sense.clear()
        self.timestamp_fmt = "%Y-%m-%d %H:%M:%S"
        self.measurement_interval = 30.

        # Read sensor values ones to make sure that they are available for recording
        test = self.sense.get_humidity()
        test = self.sense.get_temperature_from_humidity()
        test = self.sense.get_pressure()

        # Say hello
        self.sense.show_message("Starting up...")

    def compute_ambient_values(self):
        self.ambient_temperature = ambient_temperature(self.measured_temperature)
        self.ambient_pressure = ambient_pressure(self.measured_pressure)
        self.ambient_humidity = ambient_humidity(self.measured_humidity, self.measured_temperature,
                                                 self.measured_pressure, self.ambient_temperature,
                                                 self.ambient_pressure)

    def update(self):

        self.measured_humidity = self.sense.get_humidity()
        assert self.measured_humidity >= 0., "Sensor.update: humidity < 0% received from sensor."
        assert self.measured_humidity <= 100., "Sensor.update: humidity > 100% received from sensor."

        self.measured_temperature = self.sense.get_temperature_from_humidity()
        assert self.measured_temperature >= 0., "Sensor.update: temperature < 0 °C received from sensor."
        assert self.measured_temperature <= 65., "Sensor.update: temperature > 65 °C received from sensor."

        self.measured_pressure = self.sense.get_pressure()
        assert self.measured_pressure >= 260., "Sensor.update: pressure < 260 hPa received from sensor."
        assert self.measured_pressure <= 1260., "Sensor.update: pressure > 1260 hPa received from sensor."

        self.timestamp = datetime.utcnow().strftime(self.timestamp_fmt)

        self.compute_ambient_values()

    def wait(self):
        time.sleep(self.measurement_interval)

## Class ClimateDB

The class creates a table "ClimateRecord" in a sqlite3 database file and writes measurement records with timestamps.

In [36]:
class ClimateDB():

    def __init__(self, dbfile=None):
        
        assert dbfile != None, "ClimateDB: need path to database file."
        
        self.dbfile = dbfile
        conn = sqlite.connect(self.dbfile)
        cur = conn.cursor()
        cur.execute(
        """
        CREATE TABLE IF NOT EXISTS ClimateRecord (TimeStamp TEXT, Humidity REAL,
        Temperature REAL, Pressure REAL)
        """
        )
        conn.close()
    
    def write(self, timestamp, humidity, temperature, pressure):

        assert type(timestamp) is types.StringType, "ClimateDB.write: timestamp is not a string"
        assert type(humidity) is types.FloatType, "ClimateDB.write: humidity is not a number"
        assert type(temperature) is types.FloatType, "ClimateDB.write: temperature is not a number"
        assert type(pressure) is types.FloatType, "ClimateDB.write: pressure is not a number"

        values = (timestamp, humidity, temperature, pressure)

        conn = sqlite.connect(self.dbfile)
        cur = conn.cursor()
        cur.execute("INSERT INTO ClimateRecord VALUES(?,?,?,?)",
                values)
        conn.commit()
        conn.close()

## Main function

The main function initialises the Sense Hat and output database. It then enters an infinite loop and writes climate records in fixed time intervals.

In [ ]:
if __name__ == "__main__":
    
    ClimateMeas = Sensor()
    CliDB = ClimateDB("/home/pi/Documents/roomclimate.db")

    while True:
        ClimateMeas.update()
        CliDB.write(ClimateMeas.timestamp,
                    ClimateMeas.ambient_humidity,
                    ClimateMeas.ambient_temperature,
                    ClimateMeas.ambient_pressure)
        ClimateMeas.wait()